# Query Rewrite and Web Search Context for LLMs

- This code snippet demonstrates how to rewrite a query, perform a web search, and prepare the results for use with a language model (LLM). It includes functions to rewrite the query using a language model, perform a web search using the rewritten query, and format the search results for the LLM.





In [5]:
import sys
import os
from openai import AzureOpenAI
import requests
import json
from urllib.parse import urljoin

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import load_prompt
import sys
import logging
parent_dir = os.path.dirname(os.path.dirname(os.getcwd()))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from utils.search_utils import web_search
from IPython.display import Markdown, display
from datetime import datetime
import time
import os
import pytz
from dotenv import load_dotenv
load_dotenv(override=True) 

True

In [11]:
# Configure logging
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


# Get credentials from environment variables
BING_GROUNDING_PROJECT_ENDPOINT = os.getenv("BING_GROUNDING_PROJECT_ENDPOINT")
BING_GROUNDING_CONNECTION_ID = os.getenv("BING_GROUNDING_CONNECTION_ID")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_MAX_RESULTS = int(os.getenv("BING_GROUNDING_MAX_RESULTS", 10))
BING_GROUNDING_MARKET = os.getenv("BING_GROUNDING_MARKET", "ko-KR")
BING_GROUNDING_SET_LANG = os.getenv("BING_GROUNDING_SET_LANG", "ko-KR")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "gpt-4o-mini")

# Web search mode: "google" or "bing"
web_search_mode = "bing"


client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-08-01-preview"
)

current_dir = os.getcwd()

def rewrite_query_for_search(query, client: AzureOpenAI):
    
        
        prompt_path = os.path.join(current_dir, "prompts", "rewrite_prompt.yaml")
        QUERY_REWRITE_PROMPT = load_prompt(prompt_path, encoding="utf-8")
        response = client.chat.completions.create(
            model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT.format(
                  user_query=query)},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )

        print("Rewritten query for search:", response.choices[0].message.content.strip())
        
        return json.loads(response.choices[0].message.content.strip())


In [12]:

# 웹 검색 결과를 활용해 LLM 답변을 생성하는 비동기 함수
async def process_web_search(RESULTS_COUNT, input, web_search_mode=None, product_name=None):
    
    start_time = time.time()
    print(f"Original Input: {input}")

    # 검색 모드가 지정되지 않으면 환경변수
    if web_search_mode is None:
        web_search_mode = os.getenv("WEB_SEARCH_MODE", "google").lower()

    print(f"############## Web Search Mode: {web_search_mode}")
    # query rewrite (검색용/LLM용)
    query_rewrite = rewrite_query_for_search(input, client)
    print(f"Web Search Query: {query_rewrite['search_query']}")
    print(f"LLM Query: {query_rewrite['llm_query']}")

    # 웹 검색 실행 (비동기)
    print("Search rewrited search queries...")   
    results = await web_search(query_rewrite, RESULTS_COUNT, web_search_mode=web_search_mode, product_name=product_name)

    current_date = datetime.now(tz=pytz.timezone("Asia/Seoul")).strftime("%Y-%m-%d")


    # Get the current working directory for this notebook
    current_dir = os.getcwd()
    prompt_path = os.path.join(current_dir, "prompts", "generate_prompt.yaml")
    GENERATE_PROMPT = load_prompt(prompt_path, encoding="utf-8")
    
    answer_messages = [
        {"role": "system", "content": GENERATE_PROMPT.format(
            product_name=product_name,
            date=current_date,
            contexts=results if results else '검색 결과 없음',
            user_query=query_rewrite['llm_query'],
        )},
        {"role": "user", "content": query_rewrite['llm_query']}
    ]
            
    # 검색 결과가 리스트(정상)인 경우 LLM 호출, 아니면 안내 메시지 출력
    
    print("Generate response...")   
    
    if web_search_mode == "google" and isinstance(results, list) and results:
        response = client.chat.completions.create(
            model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
            messages=answer_messages,
            top_p=0.9,
            max_tokens=1500
        )
        display(Markdown(response.choices[0].message.content))
    elif web_search_mode == "bing" and isinstance(results, list) and results:
        for result in results:
            if isinstance(result, str):
                display(Markdown(result))
            else:
                display(Markdown(str(result)))

    else:
        display(Markdown("검색 결과가 없습니다. 다른 질문을 해주세요."))
    
    end_time = time.time()
    print(f"elapsed time: {end_time - start_time:.2f} seconds\n\n")


In [16]:
RESULTS_COUNT = 5

inputs = [
    "엄빠폰 느낌좋은걸로 추구미 비교 추천",
    # "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    # "삼성전자 25년 제품이 작년 대비 좋아진것은",
    # "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    # "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    # "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]

web_search_mode = "bing"

for input in inputs:
    await process_web_search(RESULTS_COUNT, input, web_search_mode, product_name="삼성전자")  # product_name은 필요에 따라 변경 가능

Original Input: 엄빠폰 느낌좋은걸로 추구미 비교 추천
############## Web Search Mode: bing


Rewritten query for search: {
"search_query": "엄빠폰 느낌 좋은 추구미 비교 추천",
"llm_query": "엄빠폰의 느낌이 좋은 추구미 모델들을 비교하여 추천해 주세요. 각 모델의 특징과 장점을 설명해 주시면 감사하겠습니다."
}
Web Search Query: 엄빠폰 느낌 좋은 추구미 비교 추천
LLM Query: 엄빠폰의 느낌이 좋은 추구미 모델들을 비교하여 추천해 주세요. 각 모델의 특징과 장점을 설명해 주시면 감사하겠습니다.
Search rewrited search queries...
Generate response...


다양한 엄빠폰 모델 중에서 추구미 모델을 비교하고 추천드리겠습니다. 최신 정보에 기반하여 각 모델의 특징과 장점을 정리했습니다.

### 1. **갤럭시 S25 Ultra**
- **가격**: 약 135만 원
- **특징**: 
  - Snapdragon 8 Gen 4 칩셋 및 12GB RAM으로 고사양 게임과 멀티태스킹에 강력합니다.
  - 200MP의 고해상도 카메라와 5배 광학 줌 기능을 지원합니다. 
  - 대형 6.8인치 QHD+ AMOLED 디스플레이로 뛰어난 화질을 자랑합니다.
- **장점**: 
  - 최고의 카메라 성능과 멀티미디어 처리 능력.
  - Samsung 생태계와의 유기적인 연동이 강력합니다【5:0†source】【5:1†source】.

### 2. **아이폰 16 Pro**
- **가격**: 약 179만 원
- **특징**: 
  - A18 Pro 칩셋과 최적화된 iOS로 매우 빠른 속도를 제공합니다.
  - 48MP 메인 카메라와 8K 영상 촬영 기능이 있습니다.
- **장점**: 
  - 뛰어난 사진 및 영상 품질, 다수의 콘텐츠 제작에 적합합니다.
  - 안드로이드와의 연동이 필요하지 않다면 탁월한 선택입니다【5:1†source】【3:1†source】.

### 3. **Xiaomi 14 Ultra**
- **가격**: 약 89만 원
- **특징**: 
  - 라이카와의 협업으로 고급 카메라 성능을 자랑하며, 5,300mAh 대용량 배터리를 탑재하고 있습니다.
  - 고급 OLED 화면을 통한 선명한 시각 경험을 제공합니다.
- **장점**: 
  - 가격 대비 뛰어난 성능을 보장해 가성비 우수한 모델입니다.
  - 고해상도 사진과 긴 배터리 수명이 장점입니다【5:0†source】【3:0†source】.

### 4. **Nothing Phone 2a**
- **가격**: 약 59만 원
- **특징**: 
  - 투명한 디자인과 순정 안드로이드 경험을 제공합니다.
  - 120Hz 디스플레이로 매끄러운 사용자 경험을 제공합니다.
- **장점**: 
  - 독특한 디자인과 뛰어난 주행성능으로 기본적인 사용을 위해서는 충분합니다【5:1†source】【3:0†source】.

### 추천 모델
- **가장 추천하는 모델**: **갤럭시 S25 Ultra**. 고급 사용자에게 적합하며 뛰어난 카메라와 성능을 제공합니다.
- **가성비 좋은 선택**: **Xiaomi 14 Ultra**. 가격이 낮으면서도 고성능을 원하는 사용자에게 적합합니다.
- **프리미엄 사용자에게**: **아이폰 16 Pro**는 iOS 환경과 우수한 카메라 성능을 갖추고 있어 만족도가 높습니다.

스마트폰은 개인의 사용 패턴과 필요에 따라 선택하는 것이 중요하므로, 본인의 우선 순위에 맞게 신중하게 결정하시기 바랍니다!📱✨ 

더욱 자세한 정보는 아래 링크를 참조하세요:
- [2025년 최신 스마트폰 비교 및 구매 가이드](https://harugun.com/entry/2025%EB%85%84-%EC%B5%9C%EC%8B%A0-%EC%8A%A4%EB%A7%88%ED%8A%B8%ED%8F%B0-%EB%B9%84%EA%B5%90-%EB%B0%8F-%EA%B5%AC%EB%A7%A4-%EA%B0%80%EC%9D%B4%EB%93%9C)【5:0†source】.
- [2025 스마트폰 추천 목록](https://artemisrich.tistory.com/entry/2025%EB%85%84-%EC%B5%9C%EC%8B%A0-%EC%8A%A4%EB%A7%88%ED%8A%B8%ED%8F%B0-%EB%B9%84%EA%B5%90-%EC%96%B4%EB%96%A4-%EB%AA%A8%EB%8D%B8%EC%9D%84-%EC%84%A0%ED%83%9D%ED%95%B4%EC%95%BC-%ED%95%A0%EA%B9%8C)【5:1†source】.

elapsed time: 31.71 seconds


